In [3]:
import pandas as pd
from datetime import datetime
from typing import Optional
import re

In [4]:
import pandas as pd
from datetime import datetime
from typing import Optional
import re

class TravelLinkGenerator:
    def __init__(self, stations_file='all_stations.csv'):
        """Initialize with the path to the stations CSV file."""
        try:
            # Read CSV with correct column names
            self.stations_df = pd.read_csv(stations_file)
            # Convert station names to lowercase for case-insensitive matching
            self.stations_df['Station Name'] = self.stations_df['Station Name'].str.lower()
            
            # Create a mapping for major cities to their main stations
            self.major_stations = {
                'lucknow': 'LKO',  # Lucknow main station
                'delhi': 'NDLS',   # New Delhi station
                'new delhi': 'NDLS',
                'old delhi': 'DLI',
                'mumbai': 'BCT',   # Mumbai Central
                'bombay': 'BCT',
                'kolkata': 'HWH',  # Howrah
                'calcutta': 'HWH',
                'chennai': 'MAS',  # Chennai Central
                'madras': 'MAS',
                'bangalore': 'SBC', # Bangalore City
                'bengaluru': 'SBC',
                'hyderabad': 'HYB', # Hyderabad Deccan
                'secunderabad': 'SC',
                'ahmedabad': 'ADI', # Ahmedabad Junction
                'pune': 'PUNE',    # Pune Junction
                'jaipur': 'JP'     # Jaipur Junction
            }
            
            # Print available major stations
            print("\nAvailable major stations:")
            for city, code in self.major_stations.items():
                print(f"{city.title()}: {code}")
            
        except FileNotFoundError:
            raise FileNotFoundError(f"Stations file {stations_file} not found. Please ensure the file exists.")

    def get_station_code(self, city_name: str) -> str:
        """Get station code from city name."""
        city_name = city_name.lower().strip()
        
        # First check if it's a major city
        if city_name in self.major_stations:
            return self.major_stations[city_name]
        
        # If not a major city, search in the database
        station = self.stations_df[self.stations_df['Station Name'].str.contains(city_name, case=False, na=False)]
        
        if station.empty:
            # If no match found, show available major stations
            raise ValueError(
                f"\nStation code not found for city: {city_name}\n"
                f"Please try one of these major cities:\n"
                f"{', '.join(sorted(self.major_stations.keys()))}"
            )
        
        # If multiple matches found, use the one with most trains
        station = station.sort_values('Trains passing through', ascending=False)
        return station.iloc[0]['Station Code']

    def _format_date(self, date_str: str) -> str:
        """Convert date from YYYY-MM-DD to DDMMYYYY format."""
        try:
            date_obj = datetime.strptime(date_str, "%Y-%m-%d")
            if date_obj < datetime.now().replace(hour=0, minute=0, second=0, microsecond=0):
                raise ValueError("Date cannot be in the past")
            return date_obj.strftime("%d%m%Y")
        except ValueError as e:
            if "time data" in str(e):
                raise ValueError("Invalid date format. Please use YYYY-MM-DD format.")
            raise e

    def generate_flight_link(
        self,
        from_city: str,
        to_city: str,
        from_date: str,
        to_date: Optional[str] = None,
        adults: int = 1,
        children: int = 0,
        infants: int = 0,
        class_type: str = "e"
    ) -> str:
        """Generate flight search link using city names."""
        # Get station codes
        from_code = self.get_station_code(from_city)
        to_code = self.get_station_code(to_city)
        
        if from_code == to_code:
            raise ValueError("Departure and arrival cities cannot be the same")

        # Validate passenger counts
        if adults < 1:
            raise ValueError("At least one adult passenger is required")
        if children < 0 or infants < 0:
            raise ValueError("Passenger counts cannot be negative")
        if adults + children + infants > 9:
            raise ValueError("Total number of passengers cannot exceed 9")
        if infants > adults:
            raise ValueError("Number of infants cannot exceed number of adults")

        # Format dates
        formatted_from_date = self._format_date(from_date)
        if to_date:
            formatted_to_date = self._format_date(to_date)
            date_param = f"{formatted_from_date}/{formatted_to_date}"
        else:
            date_param = formatted_from_date
        
        # Generate link
        base_url = "https://www.ixigo.com/search/result/flight"
        link = (
            f"{base_url}?"
            f"from={from_code}&"
            f"to={to_code}&"
            f"date={date_param}&"
            f"adults={adults}&"
            f"children={children}&"
            f"infants={infants}&"
            f"class={class_type}&"
            f"source=Search+Form"
        )
        
        return link

    def generate_train_link(
        self,
        from_city: str,
        to_city: str,
        from_date: str,
        to_date: Optional[str] = None,
        adults: int = 1,
        children: int = 0,
        seniors: int = 0,
        students: int = 0,
        class_type: str = "ALL"
    ) -> str:
        """Generate train search link using city names."""
        # Get station codes
        from_code = self.get_station_code(from_city)
        to_code = self.get_station_code(to_city)
        
        if from_code == to_code:
            raise ValueError("Departure and arrival cities cannot be the same")

        # Validate passenger counts
        if adults < 1:
            raise ValueError("At least one adult passenger is required")
        if any(count < 0 for count in [children, seniors, students]):
            raise ValueError("Passenger counts cannot be negative")
        if adults + children + seniors + students > 6:
            raise ValueError("Total number of passengers cannot exceed 6")

        # Format dates
        formatted_from_date = self._format_date(from_date)
        if to_date:
            formatted_to_date = self._format_date(to_date)
            date_param = f"{formatted_from_date}/{formatted_to_date}"
        else:
            date_param = formatted_from_date
        
        # Generate link
        base_url = "https://www.ixigo.com/search/result/train"
        link = (
            f"{base_url}/{from_code}/{to_code}/{date_param}//"
            f"{adults}/{children}/{seniors}/{students}/{class_type}"
        )
        
        return link

def get_user_input():
    """Get travel details from user input."""
    print("\n=== Travel Link Generator ===")
    print("Enter travel details (press Enter to use defaults):")
    
    from_city = input("From City (e.g., Lucknow): ").strip()
    to_city = input("To City (e.g., Delhi): ").strip()
    from_date = input("From Date (YYYY-MM-DD): ").strip()
    to_date = input("To Date (YYYY-MM-DD) [optional]: ").strip() or None
    
    # Get passenger counts
    adults = int(input("Number of adults [default: 1]: ").strip() or "1")
    children = int(input("Number of children [default: 0]: ").strip() or "0")
    infants = int(input("Number of infants [default: 0]: ").strip() or "0")
    seniors = int(input("Number of senior citizens [default: 0]: ").strip() or "0")
    students = int(input("Number of students [default: 0]: ").strip() or "0")
    
    return {
        "from_city": from_city,
        "to_city": to_city,
        "from_date": from_date,
        "to_date": to_date,
        "adults": adults,
        "children": children,
        "infants": infants,
        "seniors": seniors,
        "students": students
    }

# Example usage
if __name__ == "__main__":
    try:
        # Create generator instance
        generator = TravelLinkGenerator('all_stations.csv')
        
        # Get user input
        travel_details = get_user_input()
        
        # Generate flight link
        flight_link = generator.generate_flight_link(
            from_city=travel_details["from_city"],
            to_city=travel_details["to_city"],
            from_date=travel_details["from_date"],
            to_date=travel_details["to_date"],
            adults=travel_details["adults"],
            children=travel_details["children"],
            infants=travel_details["infants"]
        )
        print("\nGenerated Flight Search Link:")
        print(flight_link)
        
        # Generate train link
        train_link = generator.generate_train_link(
            from_city=travel_details["from_city"],
            to_city=travel_details["to_city"],
            from_date=travel_details["from_date"],
            to_date=travel_details["to_date"],
            adults=travel_details["adults"],
            children=travel_details["children"],
            seniors=travel_details["seniors"],
            students=travel_details["students"]
        )
        print("\nGenerated Train Search Link:")
        print(train_link)
        
    except ValueError as e:
        print(f"\nError: {e}")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}") 


Available major stations:
Lucknow: LKO
Delhi: NDLS
New Delhi: NDLS
Old Delhi: DLI
Mumbai: BCT
Bombay: BCT
Kolkata: HWH
Calcutta: HWH
Chennai: MAS
Madras: MAS
Bangalore: SBC
Bengaluru: SBC
Hyderabad: HYB
Secunderabad: SC
Ahmedabad: ADI
Pune: PUNE
Jaipur: JP

=== Travel Link Generator ===
Enter travel details (press Enter to use defaults):

Generated Flight Search Link:
https://www.ixigo.com/search/result/flight?from=LKO&to=NDLS&date=28042025&adults=1&children=0&infants=0&class=e&source=Search+Form

Generated Train Search Link:
https://www.ixigo.com/search/result/train/LKO/NDLS/28042025//1/0/0/0/ALL
